<a href="https://colab.research.google.com/github/mMitsu0/Esteg.IO/blob/main/Esteg.IO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pycryptodome gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 71.1 MB/s eta 0:00:00


In [7]:
import os
import struct
import zlib
import io
from typing import Optional

import numpy as np
from PIL import Image
from Crypto.Cipher import AES
from Crypto.Protocol.KDF import PBKDF2
from Crypto.Random import get_random_bytes
import gradio as gr

PBKDF2_ITERS = 100_000
SALT_LEN = 16
NONCE_LEN = 16
TAG_LEN = 16

# ============================
#  LÓGICA DE ESTEGANOGRAFIA
# ============================

def criptografar_texto(mensagem: str, senha: str) -> bytes:
    salt = get_random_bytes(SALT_LEN)
    key = PBKDF2(senha.encode('utf-8'), salt, dkLen=32, count=PBKDF2_ITERS)
    cipher = AES.new(key, AES.MODE_EAX)
    comprimido = zlib.compress(mensagem.encode('utf-8'))
    ciphertext, tag = cipher.encrypt_and_digest(comprimido)
    return salt + cipher.nonce + tag + ciphertext

def descriptografar_payload(payload: bytes, senha: str) -> str:
    if len(payload) < (SALT_LEN + NONCE_LEN + TAG_LEN):
        raise ValueError("Payload muito curto / corrompido.")
    salt = payload[:SALT_LEN]
    nonce = payload[SALT_LEN:SALT_LEN + NONCE_LEN]
    tag = payload[SALT_LEN + NONCE_LEN:SALT_LEN + NONCE_LEN + TAG_LEN]
    ciphertext = payload[SALT_LEN + NONCE_LEN + TAG_LEN:]
    key = PBKDF2(senha.encode('utf-8'), salt, dkLen=32, count=PBKDF2_ITERS)
    cipher = AES.new(key, AES.MODE_EAX, nonce=nonce)
    comprimido = cipher.decrypt_and_verify(ciphertext, tag)
    texto = zlib.decompress(comprimido).decode('utf-8')
    return texto

def bytes_para_bits_generador(data: bytes):
    for b in data:
        for i in range(7, -1, -1):
            yield (b >> i) & 1

# ============================
#  GERADORES DE IMAGEM (PIL)
# ============================

def gerar_imagem_aleatoria_pil(largura: int = 256, altura: int = 256) -> Image.Image:
    dados = np.random.randint(0, 256, (altura, largura, 3), dtype=np.uint8)
    img = Image.fromarray(dados)
    return img

def gerar_imagem_complexa_pil(largura: int = 512, altura: int = 512) -> Image.Image:
    cx = np.random.uniform(-0.8, -0.2)
    cy = np.random.uniform(-0.3, 0.3)
    zoom = np.random.uniform(0.5, 2.0)

    escala_x = 3.0 / zoom
    escala_y = 3.0 / zoom

    x = np.linspace(cx - escala_x / 2, cx + escala_x / 2, largura)
    y = np.linspace(cy - escala_y / 2, cy + escala_y / 2, altura)
    X, Y = np.meshgrid(x, y)
    C = X + 1j * Y
    Z = np.zeros_like(C, dtype=complex)

    img = np.zeros((altura, largura, 3), dtype=np.uint8)

    max_iter = np.random.randint(60, 200)

    base_r = np.random.randint(20, 256)
    base_g = np.random.randint(20, 256)
    base_b = np.random.randint(20, 256)

    for i in range(max_iter):
        mask = np.abs(Z) <= 2
        Z[mask] = Z[mask]**2 + C[mask]

        fator = (i / max_iter)
        r = (base_r * fator) % 256
        g = (base_g * (1 - fator)) % 256
        b = (base_b * (0.5 + 0.5 * fator)) % 256

        img[mask] = np.stack([
            np.full_like(Z[mask].real, int(r), dtype=np.uint8),
            np.full_like(Z[mask].real, int(g), dtype=np.uint8),
            np.full_like(Z[mask].real, int(b), dtype=np.uint8)
        ], axis=1)

    ruido_int = np.random.randint(30, 90)
    ruido = np.random.randint(0, ruido_int, (altura, largura, 3), dtype=np.uint8)
    img = np.clip(img + ruido, 0, 255).astype(np.uint8)

    return Image.fromarray(img)

def gerar_imagem_artistica_pil(largura: int = 512, altura: int = 512) -> Image.Image:
    x = np.linspace(0, 2 * np.pi, largura)
    y = np.linspace(0, 2 * np.pi, altura)
    X, Y = np.meshgrid(x, y)

    fx_r = np.random.uniform(1.5, 4.5)
    fy_r = np.random.uniform(3.0, 7.0)
    fx_g = np.random.uniform(1.0, 4.0)
    fy_g = np.random.uniform(1.0, 4.0)
    fx_b = np.random.uniform(3.0, 6.0)
    fy_b = np.random.uniform(0.5, 3.0)

    phase_r = np.random.uniform(0, 2 * np.pi)
    phase_g = np.random.uniform(0, 2 * np.pi)
    phase_b = np.random.uniform(0, 2 * np.pi)

    r = (np.sin(X * fx_r + phase_r) + np.cos(Y * fy_r)) * 127 + 128
    g = (np.sin(X * fx_g + Y * fy_g + phase_g)) * 127 + 128
    b = (np.cos(X * fx_b - Y * fy_b + phase_b)) * 127 + 128

    imagem = np.stack([r, g, b], axis=2)

    cx, cy = largura // 2, altura // 2
    Xg, Yg = np.meshgrid(np.arange(largura), np.arange(altura))
    dist = np.sqrt((Xg - cx)**2 + (Yg - cy)**2)
    grad = (1 - (dist / dist.max()))
    grad = np.clip(grad, 0, 1)
    grad_strength = np.random.uniform(0.3, 0.8)
    imagem = imagem * (1 - grad_strength + grad_strength * grad[:, :, np.newaxis])

    ruido_sigma = np.random.uniform(10, 30)
    ruido = np.random.normal(0, ruido_sigma, (altura, largura, 3))
    imagem = np.clip(imagem + ruido, 0, 255).astype(np.uint8)

    return Image.fromarray(imagem)

# ============================
#  ESTEGANOGRAFIA EM PIL
# ============================

def codificar_mensagem_em_imagem_pil(img: Image.Image, mensagem: str, senha: str) -> Image.Image:
    if img is None:
        raise ValueError("Nenhuma imagem fornecida.")

    img = img.convert("RGB")
    arr = np.array(img, dtype=np.uint8)
    flat = arr.flatten()

    payload = criptografar_texto(mensagem, senha)
    header = struct.pack(">I", len(payload))
    full = header + payload

    needed_bits = len(full) * 8
    capacity_bits = flat.size
    if needed_bits > capacity_bits:
        raise ValueError(
            f"Mensagem muito grande para esta imagem. "
            f"Capacidade (bits): {capacity_bits}, necessário: {needed_bits}"
        )

    bit_gen = bytes_para_bits_generador(full)
    for i, bit in enumerate(bit_gen):
        flat[i] = (flat[i] & 0xFE) | bit

    new_arr = flat.reshape(arr.shape).astype(np.uint8)
    return Image.fromarray(new_arr)

def decodificar_mensagem_de_imagem_pil(img: Image.Image, senha: str) -> str:
    if img is None:
        raise ValueError("Nenhuma imagem fornecida.")
    img = img.convert("RGB")
    arr = np.array(img, dtype=np.uint8)
    flat = arr.flatten()

    if flat.size < 32:
        raise ValueError("Imagem muito pequena para conter header.")

    header_bytes = bytearray()
    for j in range(4):
        b = 0
        for k in range(8):
            idx = j * 8 + k
            b = (b << 1) | (int(flat[idx]) & 1)
        header_bytes.append(b)
    payload_len = struct.unpack(">I", bytes(header_bytes))[0]

    total_needed_bits = (4 + payload_len) * 8
    if total_needed_bits > flat.size:
        raise ValueError(
            "A imagem não contém bits suficientes para o payload declarado "
            "(pode estar corrompida ou ter passado por compressão)."
        )

    payload_bytes = bytearray(payload_len)
    for i in range(payload_len):
        b = 0
        for k in range(8):
            idx = 32 + i * 8 + k
            b = (b << 1) | (int(flat[idx]) & 1)
        payload_bytes[i] = b

    texto = descriptografar_payload(bytes(payload_bytes), senha)
    return texto

# ============================
#  FUNÇÕES PARA GRADIO
# ============================

def _tmp_path(prefix: str = "img") -> str:
    os.makedirs("/tmp/stego", exist_ok=True)
    rand = np.random.randint(0, 1_000_000_000)
    return f"/tmp/stego/{prefix}_{rand}.png"

def gr_gerar(tipo, largura, altura):
    try:
        largura = int(largura)
        altura = int(altura)
        if largura <= 0 or altura <= 0:
            raise ValueError("Dimensões devem ser positivas.")
    except Exception as e:
        return None, None, f"Erro nas dimensões: {e}"

    if tipo == "aleatoria":
        img = gerar_imagem_aleatoria_pil(largura, altura)
    elif tipo == "complexa":
        img = gerar_imagem_complexa_pil(largura, altura)
    else:
        img = gerar_imagem_artistica_pil(largura, altura)

    path = _tmp_path("gerada")
    img.save(path, format="PNG", compress_level=0)

    return img, path, "Imagem gerada com sucesso (PNG)."

def gr_codificar(img_pil, mensagem, senha):
    if img_pil is None:
        return None, None, "Envie uma imagem base (pode ser PNG, JPG, WebP etc.)."

    mensagem = (mensagem or "").strip()
    senha = (senha or "").strip()
    if not mensagem:
        return None, None, "Digite a mensagem a ser escondida."
    if not senha:
        return None, None, "Digite a senha."

    try:
        out_img = codificar_mensagem_em_imagem_pil(img_pil, mensagem, senha)
        path = _tmp_path("codificada")
        out_img.save(path, format="PNG", compress_level=0)
        return out_img, path, "Imagem codificada com sucesso (salva como PNG)."
    except Exception as e:
        return None, None, f"Erro ao codificar: {e}"

def gr_decodificar(path_png, senha):
    if not path_png:
        return "", "Envie a imagem codificada em PNG (a mesma que você baixou)."

    senha = (senha or "").strip()
    if not senha:
        return "", "Digite a senha."

    try:
        img = Image.open(path_png)
        texto = decodificar_mensagem_de_imagem_pil(img, senha)
        return texto, "Mensagem decodificada com sucesso."
    except Exception as e:
        msg = str(e)
        if "MAC check failed" in msg:
            return "", "Senha errada! A senha não corresponde aos dados criptografados."
        return "", f"Erro ao decodificar: {e}"

# ============================
#  INTERFACE GRADIO
# ============================

with gr.Blocks() as demo:
    gr.Markdown("# ESTEG.IO\nEsteganografia em Imagens")

    with gr.Tabs():
        # ----- Aba Gerar Imagens -----
        with gr.Tab("Gerar imagens"):
            tipo = gr.Radio(
                ["aleatoria", "complexa", "artistica"],
                value="aleatoria",
                label="Tipo de imagem",
                info="Escolha o tipo de padrão para gerar."
            )
            largura = gr.Number(value=512, precision=0, label="Largura (px)")
            altura = gr.Number(value=512, precision=0, label="Altura (px)")
            btn_gerar = gr.Button("Gerar imagem")
            img_preview = gr.Image(label="Prévia", type="pil")
            file_png = gr.File(label="Baixar imagem gerada (PNG)")
            status_gerar = gr.Markdown()

            btn_gerar.click(
                fn=gr_gerar,
                inputs=[tipo, largura, altura],
                outputs=[img_preview, file_png, status_gerar]
            )

        # ----- Aba Codificar -----
        with gr.Tab("Codificar mensagem"):
            gr.Markdown(
                "**Passo 1:** Envie a imagem base (pode ser PNG, JPG, WebP etc.).\n"
                "**Passo 2:** Digite a mensagem e a senha.\n"
                "**Passo 3:** Baixe sempre o PNG gerado para depois decodificar."
            )
            img_base = gr.Image(label="Imagem base", type="pil")
            mensagem = gr.Textbox(label="Mensagem secreta", lines=6)
            senha_cod = gr.Textbox(label="Senha", type="password")
            btn_cod = gr.Button("Codificar na imagem")
            img_cod_preview = gr.Image(label="Prévia da imagem codificada", type="pil")
            img_cod_file = gr.File(label="Baixar imagem codificada (PNG)")
            status_cod = gr.Markdown()

            btn_cod.click(
                fn=gr_codificar,
                inputs=[img_base, mensagem, senha_cod],
                outputs=[img_cod_preview, img_cod_file, status_cod]
            )

        # ----- Aba Decodificar -----
        with gr.Tab("Decodificar mensagem"):
            gr.Markdown("Envie **a mesma imagem PNG codificada** que você baixou na aba anterior.")
            img_cod_in = gr.File(
                label="Imagem codificada (PNG)",
                file_types=[".png"],
                type="filepath"
            )
            senha_dec = gr.Textbox(label="Senha", type="password")
            btn_dec = gr.Button("Decodificar")
            texto_dec = gr.Textbox(label="Mensagem decodificada", lines=8)
            status_dec = gr.Markdown()

            btn_dec.click(
                fn=gr_decodificar,
                inputs=[img_cod_in, senha_dec],
                outputs=[texto_dec, status_dec]
            )

        # ----- Aba Sobre -----
        with gr.Tab("Sobre"):
            gr.Markdown(
                """
**Programa de Esteganografia em Imagens**

Feito pelos alunos de Ciência da Computação 8° Semestre:

- Matheus Mitsuo Tomotake Santos – RGM: 30248531
- Pedro Gavioli Pinarde – RGM: 25798006
- Lucas Félix Nogueira – RGM: 28851994
- Gabriel Souza Cavalcante – RGM: 30386730

Interface web criada para funcionar no Google Colab, mantendo toda a lógica de esteganografia
(criptografia + LSB) e garantindo download direto em PNG, sem conversão para WebP.
                """
            )

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://05308288bd6c929d7b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
